### Imports iniciales
Se realizan los siguientes imports:
 - arff para la apertura del archivo de datos
 - pandas para el manejo de datos en tablas
 - numpy para el manejo de operaciones matemáticas y manipulación de matrices, listas y arrays

In [ ]:
!pip install qgrid
!pip install scipy
!pip install sklearn

In [9]:
import qgrid


from scipy.io import arff
import pandas as pd
import numpy as np


from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline


from sklearn.neighbors import KNeighborsClassifier

In [ ]:



#####################################################################################

#ETAPA 1: Extracción de los datos desde archivo .arff obtenido de UCI
ds = arff.loadarff('PhishingData.arff')
df = pd.DataFrame(ds[0])
df.head()
dataset = df.to_numpy()
X_parcial = dataset[: , 0:9]
Y_parcial = dataset[: , 9]
X = np.zeros(len(X_parcial))
Y = np.zeros(len(Y_parcial))
j = 0
for i in Y_parcial:
    s = i.decode('UTF-8')
    Y[j] = int(s)
    j = j + 1
X_lista = list()
for i in X_parcial:
    lista = list()
    for k in i:
        s = k.decode('UTF-8')
        lista.append(int(s))
    X_lista.append(lista)
X = np.asarray(X_lista)

# ¿CUÁNTAS MUESTRAS HAY? ¿CUÁNTAS CARACTERÍSTICAS TIENE EL PROBLEMA?
muestras = len(X)
print("Hay " + str(muestras) + " muestras en el conjunto de datos.")
caracteristicas = len(X[0])
print("Hay " + str(caracteristicas) + " características en el conjunto de datos.")


 #####################################################################################


#Función para calcular error de clasificación
def error(Y_lest, Y):
    error = 0
    for ye, y in zip(Y_lest, Y):
        if ye != y:
            error += 1
    error = error/np.size(Y)
    return error


 #####################################################################################


#ETAPA 2: Ejecución de validación cruzada, clasificación k-vecinos
def k_vecinos(n_vec):
    folds = 4
    skf = StratifiedKFold(n_splits = folds)
    vec_error_clasif = np.zeros(folds)
    j = 0
    for train_index, test_index in skf.split(X , Y):
        print("TRAIN:", train_index, "TEST:", test_index)
        X_train, X_test = X[train_index], X[test_index]
        Y_train, Y_test = Y[train_index], Y[test_index]

        #En un sólo pipeline se estandarizan y se procesan los datos con el clasificador_knn
        clasificador_knn = Pipeline([('scaler', StandardScaler()),('knn', KNeighborsClassifier(n_neighbors = n_vec))])
        clasificador_knn.fit(X_train , Y_train)
        Yestim = clasificador_knn.predict(X_test)
        vec_error_clasif[j] = error(Yestim, Y_test)
        j = j + 1
    return np.mean(vec_error_clasif) , np.std(vec_error_clasif)


 #####################################################################################


# Obtención tabla para determinar número óptimo de vecinos
df_types = pd.DataFrame({'Numero de vecinos' : pd.Series(['1', '2', '3', '4', '5', '6', '7', '100'])})
df_types["Error_Prueba"] = pd.Series()
df_types["Desviación estándar del error"] = ""

numero_vecinos = [1, 2, 3, 4, 5, 6, 7, 100]
for i in range(len(numero_vecinos)):
    errorprueba, desvesterror = k_vecinos(numero_vecinos[i])
    print('\nEjecución ' + str(i + 1) + ' lista.' + '\nFaltan ' + str(len(numero_vecinos) - i - 1) + ' ejecuciones.\n')
    df_types.loc[i, "Error_Prueba"] = errorprueba # cambiar por el valor correcto
    df_types.loc[i, "Desviación estándar del error"] = desvesterror  # cambiar por el valor correcto

#df_types.loc[1, "Error_Prueba"] = "0.3630" # cambiar por el valor correcto
#df_types.loc[1, "Desviación estándar del error"] = "0.0061"  # cambiar por el valor correcto
#df_types.sort_index(inplace=True)

df_types.set_index(['Numero de vecinos'], inplace=True)
qgrid_widget = qgrid.show_grid(df_types, show_toolbar=False)
qgrid_widget.get_changed_df()
